In [1]:
# Evaluation of experiments
import matplotlib.pyplot as plt
import pandas as pd

dfExperiments45 = pd.read_csv('experiments_similarityScore_45_negativeSamples_3_2019-01-07T16-37-43.467.csv', sep=',', low_memory=False, header=None)
dfExperiments45.columns = ['Timestamp', 'Experiment', 'Class', 'TargetTable', 'Precision', 'Recall', 'F-1', 'Blocker', 'BlockingFunction', 'TrainedOn', 'TrainedModel']
dfExperiments45['Indexing Threshold'] = '0.45'

dfExperiments70 = pd.read_csv('experiments_similarityScore_70_negativeSamples_3_2019-01-07T16-33-26.859.csv', sep=',', low_memory=False, header=None)
dfExperiments70.columns = ['Timestamp', 'Experiment', 'Class', 'TargetTable', 'Precision', 'Recall', 'F-1', 'Blocker', 'BlockingFunction', 'TrainedOn', 'TrainedModel']
dfExperiments70['Indexing Threshold'] = '0.7'

In [4]:
#Keep only baselines
dfExperiments45 = dfExperiments45[~dfExperiments45['Experiment'].str.contains('transfer_')]
dfExperiments70 = dfExperiments70[~dfExperiments70['Experiment'].str.contains('transfer_')]
#display(dfExperiments)

#Compare baselines to matching rule
def compareLearnedRuleWithBaseline(dfExperiments):

    dfBaselineExperimentsCompared = pd.DataFrame()

    for targetTable in dfExperiments['TargetTable'].unique():
        dfSubSetExperiments = dfExperiments[dfExperiments['TargetTable'] == targetTable]
        dfLearnedMatchingRule = dfSubSetExperiments[dfSubSetExperiments['Experiment'].str.contains('learned_')]
        dfBaselinesMatchingRule = dfSubSetExperiments[dfSubSetExperiments['Experiment'].str.contains('baseline_')]
    
        for indexLearned, rowLearned in dfLearnedMatchingRule.iterrows():
            dfBaselineExperimentsTmpCompared = pd.DataFrame()
            for indexBaseline, rowBaseline in dfBaselinesMatchingRule.iterrows():
                newRow = rowLearned.copy()
                newRow['Precision-Delta'] = rowLearned['Precision'] - rowBaseline['Precision']
                newRow['Recall-Delta'] = rowLearned['Recall'] - rowBaseline['Recall']
                newRow['F-1-Delta'] = rowLearned['F-1'] - rowBaseline['F-1']
                newRow['Experiment'] = rowBaseline['Experiment']
            
                dfBaselineExperimentsTmpCompared = dfBaselineExperimentsTmpCompared.append(newRow, ignore_index=True)
            
            dfBaselineExperimentsTmpCompared = dfBaselineExperimentsTmpCompared.sort_values(by=['F-1-Delta']).iloc[0]
            dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.append(dfBaselineExperimentsTmpCompared)
    return dfBaselineExperimentsCompared

dfBaselineExperimentsCompared = compareLearnedRuleWithBaseline(dfExperiments45)
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.append(compareLearnedRuleWithBaseline(dfExperiments70))
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.sort_values(by=['TargetTable'])

dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.filter(items=['TargetTable', 'Indexing Threshold',  'TrainedModel', 'Precision', 'Recall',  'F-1', 'Experiment','Precision-Delta',  'Recall-Delta','F-1-Delta'])
dfBaselineExperimentsCompared['Experiment'] = dfBaselineExperimentsCompared['Experiment'].str.replace('baseline_', '').str.replace('_citation', '')
dfBaselineExperimentsCompared['Experiment'] = dfBaselineExperimentsCompared['Experiment'].str.split('-', expand=True)[0]
dfBaselineExperimentsCompared['TrainedModel'] = dfBaselineExperimentsCompared['TrainedModel'].str.replace('J48', 'DT')
dfBaselineExperimentsCompared['TrainedModel'] = dfBaselineExperimentsCompared['TrainedModel'].str.replace('SimpleLogistic', 'LR')
dfBaselineExperimentsCompared['TargetTable'] = dfBaselineExperimentsCompared['TargetTable'].str.replace('.csv', '')

dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('all', 'All')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('bag_of_words', 'BoW')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('linear_combination', 'LC')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.replace('label', 'Label')
dfBaselineExperimentsCompared = dfBaselineExperimentsCompared.round(3)
display(dfBaselineExperimentsCompared)

print(dfBaselineExperimentsCompared.to_latex(index=False))

,TargetTable,Indexing Threshold,TrainedModel,Precision,Recall,F-1,Experiment,Precision-Delta,Recall-Delta,F-1-Delta
2,acm,0.45,DT,0.879,0.935,0.906,LC,0.204,0.000,0.122
2,acm,0.45,LR,0.853,0.935,0.892,LC,0.179,0.000,0.109
2,acm,0.7,DT,0.879,0.935,0.906,LC,0.204,0.000,0.122
2,acm,0.7,LR,0.879,0.935,0.906,LC,0.204,0.000,0.122
0,all,0.45,DT,0.628,0.790,0.700,Label,0.118,-0.048,0.066
0,all,0.45,LR,0.545,0.871,0.671,Label,0.036,0.032,0.037
0,all,0.7,DT,0.685,0.806,0.741,Label,0.175,-0.032,0.107
0,all,0.7,LR,0.626,0.919,0.745,Label,0.117,0.081,0.111
0,scholar,0.45,DT,0.469,0.778,0.585,Label,0.022,-0.071,-0.000
0,scholar,0.45,LR,0.459,0.921,0.613,Label,0.013,0.071,0.028


\begin{tabular}{lllrrrlrrr}
\toprule
TargetTable & Indexing Threshold & TrainedModel &  Precision &  Recall &    F-1 & Experiment &  Precision-Delta &  Recall-Delta &  F-1-Delta \\
\midrule
        acm &               0.45 &           DT &      0.879 &   0.935 &  0.906 &         LC &            0.204 &         0.000 &      0.122 \\
        acm &               0.45 &           LR &      0.853 &   0.935 &  0.892 &         LC &            0.179 &         0.000 &      0.109 \\
        acm &                0.7 &           DT &      0.879 &   0.935 &  0.906 &         LC &            0.204 &         0.000 &      0.122 \\
        acm &                0.7 &           LR &      0.879 &   0.935 &  0.906 &         LC &            0.204 &         0.000 &      0.122 \\
        all &               0.45 &           DT &      0.628 &   0.790 &  0.700 &      Label &            0.118 &        -0.048 &      0.066 \\
        all &               0.45 &           LR &      0.545 &   0.871 &  0.671 &      Lab

In [8]:
import csv

dfExperimentsAll = dfBaselineExperimentsCompared[dfBaselineExperimentsCompared['TargetTable'].str.contains('all')]
dfExperimentsOther = dfBaselineExperimentsCompared[~dfBaselineExperimentsCompared['TargetTable'].str.contains('all')]

dfAggregatedExperiments = dfExperimentsOther.groupby(['Indexing Threshold']).mean()
dfAggregatedExperiments['Level'] = 'Other'
dfAggregatedExperimentsAll = dfExperimentsAll.groupby(['Indexing Threshold']).mean()
dfAggregatedExperimentsAll['Level'] = 'All'

dfAggregatedExperiments = dfAggregatedExperiments.append(dfAggregatedExperimentsAll)
display(dfAggregatedExperiments)
dfAggregatedExperiments.to_csv('rq1_aggregated_results_citation.csv', sep=',', encoding='utf-8', quotechar='"', quoting=csv.QUOTE_ALL)

,Precision,Recall,F-1,Precision-Delta,Recall-Delta,F-1-Delta,Level
Indexing Threshold,,,,,,,
0.45,0.6650,0.89225,0.74900,0.1045,0.0000,0.06475,Other
0.7,0.6735,0.90475,0.75775,0.1130,0.0125,0.07325,Other
0.45,0.5865,0.83050,0.68550,0.0770,-0.0080,0.05150,All
0.7,0.6555,0.86250,0.74300,0.1460,0.0245,0.10900,All


In [ ]:
#Rearrange DataFrame
def rearrange(df):
    dfAllExperimentsRearranged = pd.DataFrame()
    for index, row in df.iterrows():
        rowPrecision = row
        rowPrecision['ResultValue'] = row['Precision-Delta']
        rowPrecision['ResultValueType'] = 'Precision-Delta'
        
        dfAllExperimentsRearranged = dfAllExperimentsRearranged.append(rowPrecision)
        
        rowRecall = row
        rowRecall['ResultValue'] = row['Recall-Delta']
        rowRecall['ResultValueType'] = 'Recall-Delta'
        
        dfAllExperimentsRearranged = dfAllExperimentsRearranged.append(rowPrecision)
        
        #rowF1 = row
        #rowF1['ResultValue'] = row['F-1']
        #rowF1['ResultValueType'] = 'F-1'
        
        #dfAllExperimentsRearranged = dfAllExperimentsRearranged.append(rowF1)
    
    return dfAllExperimentsRearranged

dfAllExperimentsRearranged = rearrange(dfBaselineExperimentsCompared)

display(dfAllExperimentsRearranged)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plotResults(df, title):
    sns.set(style="whitegrid")
    fig, ax1 = plt.subplots(figsize=(20,8))
    sns.catplot(x="shortExperiment", y="ResultValue", hue="ResultValueType", data=df, height=6, kind="bar", ax=ax1)
    
    titleAll = title
    ax1.set_title(titleAll, fontweight="bold", size=20) # Title
    ax1.set_xlabel("Experiments",fontsize = 20.0)
    ax1.set_ylabel("" ,fontsize = 20.0) # Y label
    ax1.tick_params(direction='out', length=6, width=2, labelsize=20, grid_alpha=0.5)
    ax1.tick_params(axis = 'x', labelrotation=45)
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=20)


dfAllExperimentsRearranged['shortExperiment'] = dfAllExperimentsRearranged['TrainedModel'].str.cat(dfAllExperimentsRearranged['Experiment'].str.replace('_author', '').str.replace('baseline_', '').str.replace('hotels_www.',''), sep='-')
plotResults(dfAllExperimentsRearranged, 'Baseline Comparison')

In [ ]:
display(dfAllExperimentsRearranged.filter(items=['shortExperiment','Precision', 'Recall', 'F-1']).drop_duplicates(keep='first'))

In [ ]:
round(dfAllExperimentsRearranged.filter(items=['shortExperiment', 'F-1-Delta']).drop_duplicates(keep='first')['F-1-Delta'].mean(),4)